In [1]:
import h5py
import numpy as np
import bcolz
import allel
allel.__version__

'1.0.0.dev10'

In [2]:
# data from http://www.malariagen.net/data/ag1000g-phase1-ar3
callset = h5py.File('/data/coluzzi/ag1000g/data/phase1/release/AR3/variation/main/hdf5/ag1000g.phase1.ar3.pass.h5',
                    mode='r')

## GenotypeChunkedArray

In [3]:
genotype = allel.GenotypeChunkedArray(callset['3L/calldata/genotype'])
genotype

GenotypeChunkedArray((9643193, 765, 2), int8, h5py._hl.dataset.Dataset)

In [4]:
%time genotype.max()

CPU times: user 46.1 s, sys: 240 ms, total: 46.3 s
Wall time: 46.3 s


3

In [4]:
genotype_bcolzmem = genotype.copy(stop=1000000, storage='bcolzmem')
%time genotype_bcolzmem.max()

CPU times: user 2.98 s, sys: 44 ms, total: 3.02 s
Wall time: 2.35 s


3

In [5]:
genotype_bcolztmp = genotype.copy(stop=1000000, storage='bcolztmp', dir='/magnetic')
%time genotype_bcolztmp.max()

CPU times: user 2.98 s, sys: 108 ms, total: 3.09 s
Wall time: 2.4 s


3

In [6]:
genotype_bcolzmem_zlib1 = genotype.copy(stop=1000000, storage='bcolzmem_zlib1')
%time genotype_bcolzmem_zlib1.max()

CPU times: user 5.31 s, sys: 48 ms, total: 5.36 s
Wall time: 2.85 s


3

In [7]:
genotype_bcolztmp_zlib1 = genotype.copy(stop=1000000, storage='bcolztmp_zlib1', dir='/magnetic')
%time genotype_bcolztmp_zlib1.max()

CPU times: user 5.33 s, sys: 108 ms, total: 5.44 s
Wall time: 2.98 s


3

In [8]:
genotype_hdf5mem_zlib1 = genotype.copy(stop=1000000, storage='hdf5mem_zlib1')
%time genotype_hdf5mem_zlib1.max()

CPU times: user 4.28 s, sys: 0 ns, total: 4.28 s
Wall time: 4.27 s


3

In [9]:
genotype_hdf5tmp_zlib1 = genotype.copy(stop=1000000, storage='hdf5tmp_zlib1', dir='/magnetic')
%time genotype_hdf5tmp_zlib1.max()

CPU times: user 4.35 s, sys: 12 ms, total: 4.36 s
Wall time: 4.36 s


3

### Tune default HDF5 chunk size

In [14]:
genotype.copy(storage='bcolzmem').chunklen

2741

In [15]:
genotype_bcolzmem.chunklen

1370

In [12]:
genotype_hdf5mem_zlib1.chunks

(685, 765, 2)

In [11]:
for chunksize in 2**15, 2**16, 2**17, 2**18, 2**19, 2**20, 2**21, 2**22, 2**23:
    chunklen = chunksize // (genotype.shape[1] * genotype.shape[2])
    chunks = (chunklen,) + genotype.shape[1:]
    print(chunksize, chunklen)
    genotype_hdf5mem_zlib1_bigchunks = genotype.copy(stop=1000000, storage='hdf5mem_zlib1', chunks=chunks)
    %time genotype_hdf5mem_zlib1_bigchunks.max()

32768 21
CPU times: user 11.7 s, sys: 0 ns, total: 11.7 s
Wall time: 11.7 s
65536 42
CPU times: user 8.16 s, sys: 0 ns, total: 8.16 s
Wall time: 8.15 s
131072 85
CPU times: user 5.77 s, sys: 0 ns, total: 5.77 s
Wall time: 5.77 s
262144 171
CPU times: user 4.83 s, sys: 4 ms, total: 4.84 s
Wall time: 4.83 s
524288 342
CPU times: user 4.44 s, sys: 0 ns, total: 4.44 s
Wall time: 4.44 s
1048576 685
CPU times: user 4.28 s, sys: 0 ns, total: 4.28 s
Wall time: 4.27 s
2097152 1370
CPU times: user 4.14 s, sys: 0 ns, total: 4.14 s
Wall time: 4.14 s
4194304 2741
CPU times: user 4.1 s, sys: 0 ns, total: 4.1 s
Wall time: 4.09 s
8388608 5482
CPU times: user 4 s, sys: 0 ns, total: 4 s
Wall time: 3.99 s


### Excercise methods

In [11]:
%time genotype_bcolzmem.is_called()

CPU times: user 6.88 s, sys: 116 ms, total: 6.99 s
Wall time: 2.88 s


ChunkedArray((1000000, 765), bool, bcolz.carray_ext.carray)

In [13]:
%time genotype_hdf5mem_zlib1.is_called(storage='hdf5mem_zlib1')

CPU times: user 10.9 s, sys: 88 ms, total: 11 s
Wall time: 7.53 s


ChunkedArray((1000000, 765), bool, h5py._hl.dataset.Dataset)

In [15]:
%time genotype_bcolzmem.count_het()

CPU times: user 9.38 s, sys: 76 ms, total: 9.46 s
Wall time: 3.77 s


28029171

In [16]:
%time genotype_hdf5mem_zlib1.count_het()

CPU times: user 11.2 s, sys: 72 ms, total: 11.3 s
Wall time: 6.3 s


28029171

In [17]:
ac = genotype_bcolzmem.count_alleles()
ac

AlleleCountsChunkedArray((1000000, 4), int32, bcolz.carray_ext.carray)

In [18]:
ac[:]

AlleleCountsArray((1000000, 4), dtype=int32)
[[1527    3    0    0]
 [1529    1    0    0]
 [1528    2    0    0]
 ..., 
 [1443   85    2    0]
 [1500   30    0    0]
 [1529    1    0    0]]

In [19]:
acs = genotype_bcolzmem.count_alleles_subpops(subpops={'pop1': list(range(100)),
                                                       'pop2': list(range(100, 200))},
                                              max_allele=3)
acs

ValueError: `cols` input is not supported

## Sysadmin

In [17]:
!yes | pip3 uninstall scikit-allel

Uninstalling scikit-allel-1.0.0.dev10:
  /usr/local/lib/python3.4/dist-packages/scikit_allel-1.0.0.dev10-py3.4-linux-x86_64.egg
Proceed (y/n)?   Successfully uninstalled scikit-allel-1.0.0.dev10
yes: standard output: Broken pipe
yes: write error


In [18]:
!cd /src/github/cggh/scikit-allel/ && python3 setup.py install

running install
Checking .pth file support in /usr/local/lib/python3.4/dist-packages/
/usr/bin/python3 -E -c pass
TEST PASSED: /usr/local/lib/python3.4/dist-packages/ appears to support .pth files
running bdist_egg
running egg_info
writing dependency_links to scikit_allel.egg-info/dependency_links.txt
writing scikit_allel.egg-info/PKG-INFO
writing top-level names to scikit_allel.egg-info/top_level.txt
reading manifest file 'scikit_allel.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'scikit_allel.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
copying ./allel/model/chunked/__init__.py -> build/lib.linux-x86_64-3.4/allel/model/chunked
copying ./allel/model/chunked/ext.py -> build/lib.linux-x86_64-3.4/allel/model/chunked
copying ./allel/model/chunked/storage_hdf5.py -> build/lib.linux-x86_64-3.4/allel/model/chunked
copying ./allel/model/chunked/core.py -> build/lib.linux-x86_64-3.4/all